In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import cmocean
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy.io import fits
import astropy.units as u
import scipy.stats as ss

In [ ]:
import sys
sys.path.append("..")
import panco2 as p2

# C1

## C1 with Planck

In [ ]:
path = "./results/C1/Planck"
ppf = p2.PressureProfileFitter(
    f"{path}/input_map.fits",
    1, 5,
    0.05, 9e14,
    map_size=4.0 * 60,
    coords_center=SkyCoord("12h00m00s +00d00m00s")
)

pix_kpc, half_map_kpc = ppf.cluster.arcsec2kpc(ppf.pix_size), ppf.cluster.arcsec2kpc(ppf.map_size * 60 / 2)
beam_kpc = ppf.cluster.arcsec2kpc(600.0)
r_bins = np.concatenate(([pix_kpc], np.logspace(np.log10(beam_kpc), np.log10(1.1 * half_map_kpc), 4)))

ppf.define_model("binned", r_bins)

ppf.add_filtering(beam_fwhm=600.0)

P_bins = p2.utils.gNFW(r_bins, *ppf.cluster.A10_params)
ppf.define_priors(
    P_bins=[ss.loguniform(0.01 * P, 100.0 * P) for P in P_bins],
    conv=ss.norm(1.0, 0.05),
    zero=ss.norm(0.0, 1e-5)
)
ppf.dump_to_file(f"{path}/C1_planck.panco2")

plt.close("all")
fig, ax = plt.subplots()
ax.loglog(r_bins, P_bins, "o-")
for x in [pix_kpc, beam_kpc / 2, beam_kpc, half_map_kpc, half_map_kpc * np.sqrt(2)]:
    ax.axvline(x, 0, 1, color="k", ls=":")
ax.axvline(ppf.cluster.R_500, 0, 1, color="k", ls="--")

par_vec = np.append(P_bins, [1.0, 0.0])
_ = p2.results.plot_data_model_residuals(ppf, par_vec, smooth=0.5, cbar_fact=1e5, cbar_label="Compton $y \\times 10^5$")
_, ax = p2.results.plot_data_model_residuals_1d(ppf, par_vec=par_vec, y_fact=1e5, y_label="Compton $y \\times 10^5$")
ax.set_xscale("log")
ax.set_xlim(ppf.pix_size / 2, ppf.map_size * 60 / np.sqrt(2))

In [ ]:
_ = ppf.run_mcmc(
    30, 1e5, 4, n_check=1e3, max_delta_tau=0.02, min_autocorr_times=20, 
    out_chains_file=f"{path}/rawchains.npz", plot_convergence=f"{path}/mcmc_convergence.pdf"
)

In [ ]:
path = "./results/C1/Planck"
ppf = p2.PressureProfileFitter.load_from_file(f"{path}/C1_planck.panco2")

chains_clean = p2.results.load_chains(f"{path}/rawchains.npz", 500, 50, clip_percent=25.0, verbose=True)

In [ ]:
plt.close("all")
_ = p2.results.mcmc_trace_plot(chains_clean, filename=f"{path}/mcmc_trace.png")

In [ ]:
_ = p2.results.mcmc_corner_plot(chains_clean, ppf=ppf, filename=f"{path}/mcmc_corner.pdf")

In [ ]:
meds = dict(chains_clean.median())
p2.results.plot_data_model_residuals(
    ppf, par_dic=meds, smooth=0.5, cbar_fact=1e5, lims="sym",
    cbar_label="Compton $y \\times 10^5$", filename=f"{path}/data_model_residuals_maps.pdf",
    cmap=p2.utils.get_planck_cmap()
)
#p2.results.plot_data_model_residuals_1d(
#    ppf, chains_clean=chains_clean, y_fact=1e5, plot_beam=True, 
#    y_label="Compton $y \\times 10^5$", filename=f"{path}/data_model_residuals_profiles.pdf", x_log=True
#)

In [ ]:
r_range = np.logspace(np.log10(ppf.cluster.arcsec2kpc(ppf.pix_size / 2)), np.log10(ppf.cluster.arcsec2kpc(ppf.map_size * 60 / np.sqrt(2))), 100)
fig, ax = p2.results.plot_profile(chains_clean, ppf, r_range=r_range, label="panco2")
ax.plot(r_range, p2.utils.gNFW(r_range, *ppf.cluster.A10_params), "k--", label="Truth")
ax.legend(frameon=False)
fig.savefig(f"{path}/pressure_profile.pdf")

## C1 with SPT

In [ ]:
path = "./results/C1/SPT/"
ppf = p2.PressureProfileFitter(
    f"{path}/input_map.fits",
    1, 5,
    0.05, 9e14,
    map_size=1.0 * 60,
    coords_center=SkyCoord("12h00m00s +00d00m00s")
)
ppf.sz_rms *= 2

pix_kpc, half_map_kpc = ppf.cluster.arcsec2kpc(ppf.pix_size), ppf.cluster.arcsec2kpc(ppf.map_size * 60 / 2)
beam_kpc = ppf.cluster.arcsec2kpc(75.0)
r_bins = np.concatenate(([pix_kpc], np.logspace(np.log10(beam_kpc), np.log10(1.1 * half_map_kpc), 4)))

ppf.define_model("binned", r_bins)

ppf.add_filtering(beam_fwhm=75.0)

P_bins = p2.utils.gNFW(r_bins, *ppf.cluster.A10_params)
ppf.define_priors(
    P_bins=[ss.loguniform(0.01 * P, 100.0 * P) for P in P_bins],
    conv=ss.norm(1.0, 0.05),
    zero=ss.norm(0.0, 1e-5)
)
ppf.dump_to_file(f"{path}/C1_spt.panco2")

plt.close("all")
fig, ax = plt.subplots()
ax.loglog(r_bins, P_bins, "o-")
for x in [pix_kpc, beam_kpc / 2, beam_kpc, half_map_kpc, half_map_kpc * np.sqrt(2)]:
    ax.axvline(x, 0, 1, color="k", ls=":")
ax.axvline(ppf.cluster.R_500, 0, 1, color="k", ls="--")

par_vec = np.append(P_bins, [1.0, 0.0])
_ = p2.results.plot_data_model_residuals(ppf, par_vec, smooth=1.0, cbar_fact=1e5, cbar_label="Compton $y \\times 10^5$")
_, ax = p2.results.plot_data_model_residuals_1d(ppf, par_vec=par_vec, y_fact=1e5, y_label="Compton $y \\times 10^5$", x_log=True)

In [ ]:
_ = ppf.run_mcmc(
    30, 1e5, 4, n_check=1e3, max_delta_tau=0.02, min_autocorr_times=20, 
    out_chains_file=f"{path}/rawchains.npz", plot_convergence=f"{path}/mcmc_convergence.pdf"
)

In [ ]:
path = "./results/C1/SPT/"
ppf = p2.PressureProfileFitter.load_from_file(f"{path}/C1_spt.panco2")
chains_clean = p2.results.load_chains(f"{path}/rawchains.npz", 500, 20, clip_percent=25.0, verbose=True)

In [ ]:
plt.close("all")
_ = p2.results.mcmc_trace_plot(chains_clean, filename=f"{path}/mcmc_trace.png")

In [ ]:
_ = p2.results.mcmc_corner_plot(chains_clean, ppf=ppf, filename=f"{path}/mcmc_corner.pdf")

In [ ]:
meds = dict(chains_clean.median())
p2.results.plot_data_model_residuals(
    ppf, par_dic=meds, smooth=1.0, cbar_fact=1e5, cmap="twilight_shifted",
    cbar_label="Compton $y \\times 10^5$", filename=f"{path}/data_model_residuals_maps.pdf"
)
#p2.results.plot_data_model_residuals_1d(
#    ppf, chains_clean=chains_clean, y_fact=1e5, plot_beam=True, 
#    y_label="Compton $y \\times 10^5$", filename=f"{path}/data_model_residuals_profiles.pdf", x_log=True
#)

In [ ]:
r_range = np.logspace(np.log10(ppf.cluster.arcsec2kpc(ppf.pix_size)), np.log10(ppf.cluster.arcsec2kpc(ppf.map_size * 60 / np.sqrt(2))), 100)
fig, ax = p2.results.plot_profile(chains_clean, ppf, r_range=r_range, label="panco2")
ax.plot(r_range, p2.utils.gNFW(r_range, *ppf.cluster.A10_params), "k--", label="Truth")
ax.legend(frameon=False)
fig.savefig(f"{path}/pressure_profile.pdf")

# C2

## C2 with SPT

In [ ]:
path = "./results/C2/SPT/"
ppf = p2.PressureProfileFitter(
    f"{path}/input_map.fits",
    1, 5,
    0.5, 6e14,
    map_size=1.0 * 60,
    coords_center=SkyCoord("12h00m00s +00d00m00s")
)
ppf.sz_rms *= 2

pix_kpc, half_map_kpc = ppf.cluster.arcsec2kpc(ppf.pix_size), ppf.cluster.arcsec2kpc(ppf.map_size * 60 / 2)
beam_kpc = ppf.cluster.arcsec2kpc(75.0)
r_bins = np.concatenate(([pix_kpc], np.logspace(np.log10(beam_kpc), np.log10(1.1 * half_map_kpc), 4)))

ppf.define_model("binned", r_bins)

ppf.add_filtering(beam_fwhm=75.0)

P_bins = p2.utils.gNFW(r_bins, *ppf.cluster.A10_params)
ppf.define_priors(
    P_bins=[ss.loguniform(0.01 * P, 100.0 * P) for P in P_bins],
    conv=ss.norm(1.0, 0.05),
    zero=ss.norm(0.0, 1e-5)
)
ppf.dump_to_file(f"{path}/C2_spt.panco2")

plt.close("all")
fig, ax = plt.subplots()
ax.loglog(r_bins, P_bins, "o-")
for x in [pix_kpc, beam_kpc / 2, beam_kpc, half_map_kpc, half_map_kpc * np.sqrt(2)]:
    ax.axvline(x, 0, 1, color="k", ls=":")
ax.axvline(ppf.cluster.R_500, 0, 1, color="k", ls="--")

par_vec = np.append(P_bins, [1.0, 0.0])
_ = p2.results.plot_data_model_residuals(ppf, par_vec, smooth=1.0, cbar_fact=1e5, cbar_label="Compton $y \\times 10^5$")
_, ax = p2.results.plot_data_model_residuals_1d(ppf, par_vec=par_vec, y_fact=1e5, y_label="Compton $y \\times 10^5$", x_log=True)

In [ ]:
_ = ppf.run_mcmc(
    30, 1e5, 4, n_check=1e3, max_delta_tau=0.02, min_autocorr_times=20, 
    out_chains_file=f"{path}/rawchains.npz", plot_convergence=f"{path}/mcmc_convergence.pdf"
)

In [ ]:
path = "./results/C2/SPT/"
ppf = p2.PressureProfileFitter.load_from_file(f"{path}/C2_spt.panco2")
chains_clean = p2.results.load_chains(f"{path}/rawchains.npz", 500, 50, clip_percent=20.0, verbose=True)

In [ ]:
plt.close("all")
_ = p2.results.mcmc_trace_plot(chains_clean, filename=f"{path}/mcmc_trace.png")

In [ ]:
_ = p2.results.mcmc_corner_plot(chains_clean, ppf=ppf, filename=f"{path}/mcmc_corner.pdf")

In [ ]:
meds = dict(chains_clean.median())
p2.results.plot_data_model_residuals(
    ppf, par_dic=meds, smooth=1.0, cbar_fact=1e5, cmap="twilight_shifted",
    cbar_label="Compton $y \\times 10^5$", filename=f"{path}/data_model_residuals_maps.pdf"
)
#p2.results.plot_data_model_residuals_1d(
#    ppf, chains_clean=chains_clean, y_fact=1e5, plot_beam=True, 
#    y_label="Compton $y \\times 10^5$", filename=f"{path}/data_model_residuals_profiles.pdf", x_log=True
#)

In [ ]:
r_range = np.logspace(np.log10(ppf.cluster.arcsec2kpc(ppf.pix_size)), np.log10(ppf.cluster.arcsec2kpc(ppf.map_size * 60 / np.sqrt(2))), 100)
fig, ax = p2.results.plot_profile(chains_clean, ppf, r_range=r_range, label="panco2")
ax.plot(r_range, p2.utils.gNFW(r_range, *ppf.cluster.A10_params), "k--", label="Truth")
ax.legend(frameon=False)
fig.savefig(f"{path}/pressure_profile.pdf")

## C2 with NIKA2

In [ ]:
path = "./results/C2/NIKA2/"
ppf = p2.PressureProfileFitter(
    f"{path}/input_map.fits",
    1, 5,
    0.5, 6e14,
    map_size=6.5,
    coords_center=SkyCoord("12h00m00s +00d00m00s")
)

pix_kpc, half_map_kpc = ppf.cluster.arcsec2kpc(ppf.pix_size), ppf.cluster.arcsec2kpc(ppf.map_size * 60 / 2)
beam_kpc = ppf.cluster.arcsec2kpc(18.0)
r_bins = np.concatenate(([pix_kpc], np.logspace(np.log10(beam_kpc), np.log10(1.1 * half_map_kpc), 4)))

ppf.define_model(r_bins)

tf = Table.read("./example_data/NIKA2/nk2_tf.fits")
ppf.add_filtering(beam_fwhm=18.0, k=tf["k"], tf=tf["tf_2mm"], pad=20)

P_bins = p2.utils.gNFW(r_bins, *ppf.cluster.A10_params)
ppf.define_priors(
    P_bins=[ss.loguniform(0.01 * P, 100.0 * P) for P in P_bins],
    conv=ss.norm(-12.0, 1.2),
    zero=ss.norm(0.0, 1e-3)
)
ppf.dump_to_file(f"{path}/C2_nk2.panco2")

plt.close("all")
fig, ax = plt.subplots()
ax.loglog(r_bins, P_bins, "o-")
for x in [pix_kpc, beam_kpc / 2, beam_kpc, half_map_kpc, half_map_kpc * np.sqrt(2)]:
    ax.axvline(x, 0, 1, color="k", ls=":")
ax.axvline(ppf.cluster.R_500, 0, 1, color="k", ls="--")

par_vec = np.append(P_bins, [-12.0, 0.0])
_ = p2.results.plot_data_model_residuals(ppf, par_vec, smooth=1.0, cbar_fact=1e3, cbar_label="NIKA2 150 GHz [mJy/beam]", lims="sym")
_, ax = p2.results.plot_data_model_residuals_1d(ppf, par_vec=par_vec, y_fact=-1e3, y_label="NIKA2 150 GHz [mJy/beam]", x_log=True)

In [ ]:
_ = ppf.run_mcmc(
    30, 1e5, 4, n_check=1e3, max_delta_tau=0.02, min_autocorr_times=20, 
    out_chains_file=f"{path}/rawchains.npz", plot_convergence=f"{path}/mcmc_convergence.pdf"
)

In [ ]:
path = "./results/C2/NIKA2/"
ppf = p2.PressureProfileFitter.load_from_file(f"{path}/C2_nk2.panco2")
chains_clean = p2.results.load_chains(f"{path}/rawchains.npz", 500, 20, clip_percent=20.0, verbose=True)

In [ ]:
plt.close("all")
_ = p2.results.mcmc_trace_plot(chains_clean, filename=f"{path}/mcmc_trace.png")

In [ ]:
_ = p2.results.mcmc_corner_plot(chains_clean, ppf=ppf, filename=f"{path}/mcmc_corner.pdf")

In [ ]:
meds = dict(chains_clean.median())
p2.results.plot_data_model_residuals(
    ppf, par_dic=meds, smooth=1.0, cbar_fact=1e3, cmap="RdBu_r", lims="sym",
    cbar_label="NIKA2 150 GHz [mJy/beam]", filename=f"{path}/data_model_residuals_maps.pdf"
)
#p2.results.plot_data_model_residuals_1d(
#    ppf, par_dic=meds, y_fact=-1e3, plot_beam=True, 
#    y_label="NIKA2 150 GHz [mJy/beam]", filename=f"{path}/data_model_residuals_profiles.pdf", x_log=True
#)

In [ ]:
r_range = np.logspace(np.log10(ppf.cluster.arcsec2kpc(ppf.pix_size)), np.log10(ppf.cluster.arcsec2kpc(ppf.map_size * 60 / np.sqrt(2))), 100)
fig, ax = p2.results.plot_profile(chains_clean, ppf, r_range=r_range, label="panco2")
ax.plot(r_range, p2.utils.gNFW(r_range, *ppf.cluster.A10_params), "k--", label="Truth")
ax.legend(frameon=False)
fig.savefig(f"{path}/pressure_profile.pdf")

# C3

## C3 with NIKA2

In [ ]:
path = "./results/C3/NIKA2/"
ppf = p2.PressureProfileFitter(
    f"{path}/input_map.fits",
    1, 5,
    1.0, 3e14,
    map_size=6.5,
    coords_center=SkyCoord("12h00m00s +00d00m00s")
)

pix_kpc, half_map_kpc = ppf.cluster.arcsec2kpc(ppf.pix_size), ppf.cluster.arcsec2kpc(ppf.map_size * 60 / 2)
beam_kpc = ppf.cluster.arcsec2kpc(18.0)
r_bins = np.concatenate(([pix_kpc], np.logspace(np.log10(beam_kpc), np.log10(1.1 * half_map_kpc), 4)))

ppf.define_model(r_bins)

tf = Table.read("./example_data/NIKA2/nk2_tf.fits")
ppf.add_filtering(beam_fwhm=18.0, k=tf["k"], tf=tf["tf_2mm"], pad=20)

P_bins = p2.utils.gNFW(r_bins, *ppf.cluster.A10_params)
ppf.define_priors(
    P_bins=[ss.loguniform(0.01 * P, 100.0 * P) for P in P_bins],
    conv=ss.norm(-12.0, 1.2),
    zero=ss.norm(0.0, 1e-3)
)
ppf.dump_to_file(f"{path}/C2_nk2.panco2")

plt.close("all")
fig, ax = plt.subplots()
ax.loglog(r_bins, P_bins, "o-")
for x in [pix_kpc, beam_kpc / 2, beam_kpc, half_map_kpc, half_map_kpc * np.sqrt(2)]:
    ax.axvline(x, 0, 1, color="k", ls=":")
ax.axvline(ppf.cluster.R_500, 0, 1, color="k", ls="--")

par_vec = np.append(P_bins, [-12.0, 0.0])
_ = p2.results.plot_data_model_residuals(ppf, par_vec, smooth=1.0, cbar_fact=1e3, cbar_label="NIKA2 150 GHz [mJy/beam]", lims="sym")
_, ax = p2.results.plot_data_model_residuals_1d(ppf, par_vec=par_vec, y_fact=-1e3, y_label="NIKA2 150 GHz [mJy/beam]", x_log=True)

In [ ]:
_ = ppf.run_mcmc(
    30, 1e5, 4, n_check=1e3, max_delta_tau=0.02, min_autocorr_times=20, 
    out_chains_file=f"{path}/rawchains.npz", plot_convergence=f"{path}/mcmc_convergence.pdf"
)

In [ ]:
path = "./results/C3/NIKA2/"
ppf = p2.PressureProfileFitter.load_from_file(f"{path}/C2_nk2.panco2")
chains_clean = p2.results.load_chains(f"{path}/rawchains.npz", 500, 20, clip_percent=20.0, verbose=True)

In [ ]:
plt.close("all")
_ = p2.results.mcmc_trace_plot(chains_clean, filename=f"{path}/mcmc_trace.png")

In [ ]:
_ = p2.results.mcmc_corner_plot(chains_clean, ppf=ppf, filename=f"{path}/mcmc_corner.pdf")

In [ ]:
meds = dict(chains_clean.median())
p2.results.plot_data_model_residuals(
    ppf, par_dic=meds, smooth=1.0, cbar_fact=1e3, cmap="RdBu_r", lims="sym",
    cbar_label="NIKA2 150 GHz [mJy/beam]", filename=f"{path}/data_model_residuals_maps.pdf"
)
#p2.results.plot_data_model_residuals_1d(
#    ppf, chains_clean=chains_clean, y_fact=-1e3, plot_beam=True, 
#    y_label="NIKA2 150 GHz [mJy/beam]", filename=f"{path}/data_model_residuals_profiles.pdf", x_log=True
#)

In [ ]:
r_range = np.logspace(np.log10(ppf.cluster.arcsec2kpc(ppf.pix_size)), np.log10(ppf.cluster.arcsec2kpc(ppf.map_size * 60 / np.sqrt(2))), 100)
fig, ax = p2.results.plot_profile(chains_clean, ppf, r_range=r_range, label="panco2")
ax.plot(r_range, p2.utils.gNFW(r_range, *ppf.cluster.A10_params), "k--", label="Truth")
ax.legend(frameon=False)
fig.savefig(f"{path}/pressure_profile.pdf")